In [1]:
from csv import DictReader
from datetime import datetime

from clickhouse_driver import Client


def iter_csv(filename):
    converters = {
        'qty': int,
        'time': lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    }

    with open(filename, 'r') as f:
        reader = DictReader(f)
        for line in reader:
            yield {k: (converters[k](v) if k in converters else v) for k, v in line.items()}

client = Client('localhost')

client.execute(
    'CREATE TABLE IF NOT EXISTS data_csv '
    '('
        'time DateTime, '
        'order String, '
        'qty Int32'
    ') Engine = Memory'
)

client.execute('INSERT INTO data_csv VALUES', iter_csv('data.csv'))

3

In [2]:
client.query_dataframe('SELECT * FROM data_csv')

,time,order,qty
0,2019-08-01 15:23:14,New order1,5
1,2019-08-05 09:14:45,New order2,3
2,2019-08-13 12:20:32,New order3,7


In [3]:
client.query_dataframe('show tables')

,name
0,cart_info
1,data_csv
2,dept
3,email
4,emp
5,hits
6,order_info
7,sales
8,student
9,student2
